# 자연어 처리

## 준비단계

In [2]:
import pandas as pd
import tensorflow as tf
import keras
from konlpy.tag import Mecab
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
mecab = Mecab("C:\\mecab\\mecab-ko-dic")
#fm.findSystemFonts()
# plt.rcParams['font.family']= ["NanumGothicCoding"]
# plt.rcParams["axes.unicode_minus"]=False


In [3]:
#mecab 예제
out1 = mecab.morphs("오늘은 맑은 날씨이다.")
out2 = mecab.nouns("오늘은 맑은 날씨이다.")
print(out1,out2)

['오늘', '은', '맑', '은', '날씨', '이', '다', '.'] ['오늘', '날씨']


## 데이터 전처리

In [4]:
#데이터 불러오기
train=pd.read_csv("../데이터 저장소/NaverNew_labeling105.csv")
train=train.dropna(axis=0)
train=train.reset_index(drop=True)

In [5]:
train.head(2)

,내용,Date,KOSPI,yesterdayup_down,KOSPI_shift_mean,updown,label
0,투자자라면 꼭 봐야 할 오늘의 10대 투자뉴스-4일 \n문형표 국민연금공단 이사장...,2016-01-04,1918.760010,1,1892.893994,-25.866016,0
1,"중국 증시 폭락, 한국 영향은?...1월 8일 중국 대주주 매도제한 해제 주목 \...",2016-01-05,1930.530029,0,1892.650000,-37.880029,0


### 형태소 추출

In [34]:
train["내용"]=pd.Series(map(lambda x:" ".join(mecab.nouns(x)) ,train["내용"]))

In [35]:
train

,내용,Date,KOSPI,yesterdayup_down,KOSPI_shift_mean,updown,label
0,투자자 투자 뉴스 일 문형 표 국민 연금 공단 이사장 지난달 일 취임식 참석 국민 ...,2016-01-04,1918.760010,1,1892.893994,-25.866016,0
1,중국 증시 폭락 한국 영향 월 일 중국 대주주 매도 해제 주목 중국 증시 폭락 한국...,2016-01-05,1930.530029,0,1892.650000,-37.880029,0
2,연초 체면 증권사 美 제조업 지수 년 만 최저 선진국 경제 개장 전 심리 확산 저점...,2016-01-06,1925.430054,0,1878.483984,-46.946070,0
3,충격 악재 불안 고조 충격 악재 불안 고조 증시 北 수소탄 실험 코스피 폭락 우려 ...,2016-01-07,1904.329956,1,1866.587988,-37.741968,0
4,굿모닝 증시 中 증시 몸살 코스피 아시아 경제 최동현 기자 연초 코스피 중국 증시 ...,2016-01-08,1917.619995,0,1866.700000,-50.919995,0
...,...,...,...,...,...,...,...
1693,적금 채권 선순위 반도체 株 주목 부동산 경매 노크 웰스 매니저 재테크 전망 주간 ...,2022-11-28,2408.270020,1,2381.822022,-26.447998,0
1694,고령 시대 효자 월 배당 주목 오늘 투자 전략 코스피 내외 하락 출발 전망 대형주 ...,2022-11-29,2433.389893,1,2377.670020,-55.719873,0
1695,월 증시 브레이크 데일리 김인경 기자 월 달 동안 미국 금리 인상 속도 조절 부상 ...,2022-11-30,2472.530029,1,2380.958008,-91.572021,0
1696,월 증시 산타랠리 가능 기사 내용 요약 지난달 코스피 경계 제한 상승 전망 서울 뉴...,2022-12-01,2479.840088,0,2378.935986,-100.904102,0


In [39]:
train=train.iloc[:1200]

### X  Y  train val 분리, test셋 정답지 분리

In [49]:
X_m_t=train["내용"]
Y_m_t=train["label"]
x_train_m, x_val_m, y_train_m, y_val_m=train_test_split(X_m_t,Y_m_t,test_size=0.2,random_state=42)

In [42]:
train.head(2)

,내용,Date,KOSPI,yesterdayup_down,KOSPI_shift_mean,updown,label
0,투자자 투자 뉴스 일 문형 표 국민 연금 공단 이사장 지난달 일 취임식 참석 국민 ...,2016-01-04,1918.760010,1,1892.893994,-25.866016,0
1,중국 증시 폭락 한국 영향 월 일 중국 대주주 매도 해제 주목 중국 증시 폭락 한국...,2016-01-05,1930.530029,0,1892.650000,-37.880029,0


## N-gram vectorize

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

NGRAM_RANGE = (1, 4) # 주변 요소 묶어서 분석 (1,4)->4개까지 묶어서 분석
TOP_K = 20000 # 최대 토큰 개수
TOKEN_MODE = 'word' #word 단어단위분석 char 문자단위분석
MIN_DOCUMENT_FREQUENCY = 4 #뭐지?

def ngram_vectorize(train_texts, train_labels, val_texts):

    kwargs = {
            'ngram_range': NGRAM_RANGE,  
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': TOKEN_MODE, 
            'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)
    x_train = vectorizer.fit_transform(train_texts)
    x_val = vectorizer.transform(val_texts)
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    return x_train, x_val

#### 형태소 추출 데이터 N-gram vectorize (test셋 추출용)

In [52]:
NGRAM_RANGE = (1, 4)
TOKEN_MODE = 'word'
MIN_DOCUMENT_FREQUENCY = 2
TOP_K = 20000
x_train_m_ngr,x_test_m_ngr=ngram_vectorize(x_train_m,y_train_m,X_m_t)

C:\Users\DM\anaconda3\envs\mecab\lib\site-packages\sklearn\feature_extraction\text.py:2060: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


## deeplearning 함수 단순 레이어쌓기

In [53]:
from tensorflow.keras import models
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

import tensorflow as tf


def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    op_units, op_activation = 1,'sigmoid'
    tf.random.set_seed(42)
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))

    for _ in range(layers-1):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=op_units,activation=op_activation))
    return model

## 학습모델 설계

In [56]:
def train_ngram_model(data,
                      learning_rate=1e-2,
                      epochs=1000,
                      batch_size=64,
                      layers=4,
                      units=128,
                      dropout_rate=0.5):

    (train_texts, train_labels), (val_texts, val_labels) = data
    num_classes = 2
    unexpected_labels = [v for v in val_labels if v not in range(num_classes)]
    if len(unexpected_labels):
        raise ValueError('Unexpected label values found in the validation set:'
                         ' {unexpected_labels}. Please make sure that the '
                         'labels in the validation set are in the same range '
                         'as training labels.'.format(
                             unexpected_labels=unexpected_labels))


    x_train, x_val = ngram_vectorize(
        train_texts, train_labels, val_texts)
    x_train=x_train.toarray()
    x_val=x_val.toarray()

    model = mlp_model(layers=layers,
                                  units=units,
                                  dropout_rate=dropout_rate,
                                  input_shape=x_train.shape[1:],
                                  num_classes=num_classes)


    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizers = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizers, loss=loss, metrics=['acc'])

    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=2)]

    history = model.fit(
            x_train,
            train_labels,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=(x_val, val_labels),
            verbose=2, 
            batch_size=batch_size)

    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))
    
    model.fit(x_train,train_labels)
              
    # pred = model.predict(a)
    pred=0
    model.save('IMDb_mlp_model.h5')
    tf.keras.utils.plot_model(model, show_shapes=True)
    return history['val_acc'][-1], history['val_loss'][-1],pred
    # return 1,2,pred

In [58]:
TOKEN_MODE = 'word'
NGRAM_RANGE = (1, 4)
MIN_DOCUMENT_FREQUENCY = 2
TOP_K = 20000
a,b,pred1=train_ngram_model(([(x_train_m,y_train_m),(x_val_m,y_val_m)]),dropout_rate=0.80,layers=3)

C:\Users\DM\anaconda3\envs\mecab\lib\site-packages\sklearn\feature_extraction\text.py:2060: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


Epoch 1/1000
15/15 - 1s - loss: 0.6974 - acc: 0.5302 - val_loss: 0.6880 - val_acc: 0.5250 - 1s/epoch - 91ms/step
Epoch 2/1000
15/15 - 1s - loss: 0.7026 - acc: 0.5521 - val_loss: 0.6802 - val_acc: 0.5708 - 660ms/epoch - 44ms/step
Epoch 3/1000
15/15 - 1s - loss: 0.6766 - acc: 0.5792 - val_loss: 0.6807 - val_acc: 0.5208 - 683ms/epoch - 46ms/step
Epoch 4/1000
15/15 - 1s - loss: 0.6396 - acc: 0.6073 - val_loss: 0.6636 - val_acc: 0.5917 - 676ms/epoch - 45ms/step
Epoch 5/1000
15/15 - 1s - loss: 0.6042 - acc: 0.6802 - val_loss: 0.6632 - val_acc: 0.5542 - 660ms/epoch - 44ms/step
Epoch 6/1000
15/15 - 1s - loss: 0.5349 - acc: 0.7531 - val_loss: 0.6491 - val_acc: 0.5917 - 677ms/epoch - 45ms/step
Epoch 7/1000
15/15 - 1s - loss: 0.4739 - acc: 0.7927 - val_loss: 0.6601 - val_acc: 0.6083 - 662ms/epoch - 44ms/step
Epoch 8/1000
15/15 - 1s - loss: 0.4303 - acc: 0.7948 - val_loss: 0.6569 - val_acc: 0.6250 - 649ms/epoch - 43ms/step
Validation accuracy: 0.625, loss: 0.6569452881813049
30/30 [===============

In [22]:
from keras.models import load_model
model = load_model('자연어처리 모델.h5')
pred=model.predict(x_test_m_ngr)

54/54 [==============================] - 0s 5ms/step


In [25]:
pd.DataFrame(pred).to_csv("자연어학습결과.csv",index=False)